In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [ ]:
url = "https://docs.google.com/spreadsheets/d/18X1VM1671d99V_yd-cnUI1j8oSG2ZgfU_q1HfOizErA/export?format=csv&id"
df = pd.read_csv(url)
df = df.fillna(0)
df.columns = df.columns.str.replace("-", "/")

In [ ]:
fig = go.Figure()

geo_data_cols = ["country", "location", "latitude", "longitude"]
geo_data_df = df[geo_data_cols]
dates_list = (
    df.columns.str.extract(r"(\d{2}/\d{2}/\d{4})", expand=False)
    .dropna()
    .unique()
    .to_list()
)

dfs_by_date = {}
for i, date in enumerate(dates_list):
    dfs_by_date[date] = (
        df.filter(like=date, axis=1)
        .rename(
            columns=lambda col: "deaths"
            if col.startswith("deaths")
            else "confirmed_cases"
        )
        .astype("uint32")
    )

In [ ]:
fig.data = []
for date, data in dfs_by_date.items():
    df = geo_data_df.join(data)
    df = df[df["confirmed_cases"] > 0]
    df["confirmed_cases_norm"] = np.log1p(df["confirmed_cases"])
    df["text"] = (
        date
        + "<br>"
        + df["country"]
        + "<br>"
        + df["location"]
        + "<br>Confirmed cases: "
        + df["confirmed_cases"].astype(str)
        + "<br>Deaths: "
        + df["deaths"].astype(str)
    )
    fig.add_trace(
        go.Scattergeo(
            name="",
            lat=df["latitude"],
            lon=df["longitude"],
            visible=False,
            hovertemplate=df["text"],
            showlegend=False,
            marker={
                "size": df["confirmed_cases_norm"] * 100,
                "color": "red",
                "opacity": 0.75,
                "sizemode": "area",
            },
        )
    )

In [ ]:
annotation_text_template = "<b>Worldwide Totals</b><br>{date}<br><br>Confirmed cases: {confirmed_cases:,d}<br>Deaths: {deaths:,d}<br>Mortality rate: {mortality_rate:.1%}"
annotation_dict = {
    "x": 0.03,
    "y": 0.35,
    "width": 150,
    "height": 110,
    "showarrow": False,
    "text": "",
    "valign": "middle",
    "visible": False,
    "bordercolor": "black",
}

steps = []
for i, data in enumerate(fig.data):
    step = {
        "method": "update",
        "args": [
            {"visible": [False] * len(fig.data)},
            {"annotations": [dict(annotation_dict) for _ in range(len(fig.data))]},
        ],
        "label": dates_list[i],
    }

    # toggle the i'th trace and annotation box to visible
    step["args"][0]["visible"][i] = True
    step["args"][1]["annotations"][i]["visible"] = True

    df = dfs_by_date[dates_list[i]]
    confirmed_cases = df["confirmed_cases"].sum()
    deaths = df["deaths"].sum()
    mortality_rate = deaths / confirmed_cases
    step["args"][1]["annotations"][i]["text"] = annotation_text_template.format(
        date=dates_list[i],
        confirmed_cases=confirmed_cases,
        deaths=deaths,
        mortality_rate=mortality_rate,
    )

    steps.append(step)


sliders = [
    {
        "active": 0,
        "currentvalue": {"prefix": "Date: "},
        "steps": steps,
        "len": 0.9,
        "x": 0.05,
    }
]

first_annotation_dict = {**annotation_dict}
first_annotation_dict.update(
    {
        "visible": True,
        "text": annotation_text_template.format(
            date="10/01/2020", confirmed_cases=44, deaths=1, mortality_rate=0.0227
        ),
    }
)
fig.update_layout(
    title="Novel Coronavirus Case Tracker",
    height=650,
    margin={"t": 50, "b": 20, "l": 20, "r": 20},
    annotations=[go.layout.Annotation(**first_annotation_dict)],
    sliders=sliders,
)
fig.data[0].visible = True  # set the first data point visible

fig

In [ ]:
# save the figure locally as an interactive HTML page
fig.update_layout(height=1000)
fig.write_html("nCoV_tracker.html")